# Ciencia de Datos - TP6

## Integrantes

- Ambroa, Nicolás - 229/13 - ambroanicolas@hotmail.com
- Gaustein, Diego - 586/09 - diego@gaustein.com.ar

In [1]:
from collections import Counter
from functools import wraps
from itertools import chain
from string import punctuation, whitespace
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
import os
import pickle
from pprint import pprint
from lxml import html
from nltk import sent_tokenize, word_tokenize, pos_tag, WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import sent_tokenize
from sklearn.metrics import mutual_info_score
from operator import itemgetter
import gensim

lemmatizer = WordNetLemmatizer()
stopwords = frozenset(stopwords.words('english'))

def load_or_call(func):
    """ Decorador auxiliar para cachear resultados en un .pickle. """
    @wraps(func)
    def wrapper(*args, **kwargs):
        filename = '{}-{}-{}.pickle'.format(func.__name__, str(args), str(kwargs))
        if os.path.exists(filename):
            with open(filename, 'rb') as f:
                return pickle.load(f)
        else:
            res = func(*args, **kwargs)
            with open(filename, 'wb') as f:
                pickle.dump(res, f)
            return res

    return wrapper

### 1.1. Levantar el corpus AP, separando cada noticia como un elemento distinto en un diccionario `(<DOCNO>: <TEXT>)`.

In [2]:
parsed = html.parse('ap/ap.txt')
documents = {}
for document in parsed.iter('doc'):
    docno, article = document.getchildren()
    documents[docno.text.strip()] = article.text.strip()

print('Cargados', len(documents), 'articulos')


Cargados 2250 articulos


### 1.2. Calcular el tamaño del vocabulario.

In [3]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    elif treebank_tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return wordnet.NOUN  # El default es NOUN

def get_words_for_document(document):
    for sentence in sent_tokenize(document):
        tagged_sentence = pos_tag(sentence.split())
        for word, pos in tagged_sentence:
            # Lematizar
            word = lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos))
            # Strip punctuation
            word = word.strip(punctuation)
            # Lowercase
            word = word.lower()
            
            # Skip stopwords and punctuation
            if len(word) > 1 and word not in stopwords:
                yield word

@load_or_call
def get_word_count():
    c = Counter()
    for document in documents.values():
        c.update(get_words_for_document(document))
    return c
        
c = get_word_count()
cantidad_de_palabras = sum(c.values())
print(cantidad_de_palabras, 'palabras,', len(c), 'palabras distintas.')

548169 palabras, 41090 palabras distintas.


### 1.3. Para las 500 palabras con más apariciones, calcular el par más asociado según la medida presentada.

In [4]:
"""
Para cada palabra, aproximamos p(palabra) con su frecuencia relativa según el counter.
Tomamos las 500 con más apariciones y recorremos el texto. Cada vez que encontremos una de estas miramos en una
ventana (n=8) cuando aparece alguna de las otras, y le sumamos uno.
Esta es nuestra estimación de p(palabra1, palabra2). Finalmente calculamos la información mutua e imprimimos los
pares más asociados.
"""

def mirar_ventana_y_buscar(palabra_a_asociar, palabras, palabras_a_buscar, indice_inicial, longitud_de_ventana, asociaciones):
    # Arreglo los índices para no pasarme en caso de una ventana muy cercana a los extremos de la lista.
    if indice_inicial - longitud_de_ventana < 0:
        indice_inicial = 0
    else:
        indice_inicial -= longitud_de_ventana
    if indice_inicial + longitud_de_ventana > len(palabras):
        indice_final = len(palabras)
    else:
        indice_final = indice_inicial + longitud_de_ventana
       
    # Recorro los indices, buscando palabras a buscar que no sean la palabra a asociar.
    for indice in range(indice_inicial, indice_final):
        if palabras[indice] in palabras_a_buscar and palabras[indice] != palabra_a_asociar:
            subindice_dict = str(palabra_a_asociar) + " " +str(palabras[indice])
            # Si la encontre, aumento en uno la asociación entre ambas palabras.
            try:
                asociaciones[subindice_dict] += 1
            except IndexError:
                asociaciones[subindice_dict] = 1

@load_or_call
def contar_asociaciones_de_palabras():
    contador_asociaciones_de_palabras = Counter()
    # Recorremos el texto.
    for document in documents.values():
        palabras = document.split()
        for indice, palabra in enumerate(palabras):
            #Cada vez que encontremos una de ellas, miramos en una ventana de n=8.
            if palabra in palabras_con_mas_apariciones:
                mirar_ventana_y_buscar(palabra, palabras, palabras_con_mas_apariciones, indice, 8, contador_asociaciones_de_palabras)
    return contador_asociaciones_de_palabras

# Tomamos las 500 palabras con más apariciones.
palabras_y_apariciones = c.most_common(500)
palabras_con_mas_apariciones = [tupla[0] for tupla in palabras_y_apariciones]

asociaciones_de_palabras = contar_asociaciones_de_palabras()
suma_de_asociaciones_totales = sum(asociaciones_de_palabras.values())
informaciones_mutuas = Counter()
# Recorremos los pares y calculamos información mutua para cada uno.
for palabras, asociaciones in asociaciones_de_palabras.items():
    palabras_parseadas = palabras.split(" ")
    palabra_x = palabras_parseadas[0]
    palabra_y = palabras_parseadas[1]
    proba_x = c[palabra_x] / float(cantidad_de_palabras)
    proba_y = c[palabra_y] / float(cantidad_de_palabras)
    proba_conj_x_y  = asociaciones_de_palabras[palabras] / suma_de_asociaciones_totales
    inf_mutua_x_y = proba_conj_x_y / (proba_x*proba_y)
    informaciones_mutuas[palabras] = inf_mutua_x_y

# Finalmente imprimimos los 5 pares más asociados según la información mutua
print("Los 5 pares de palabras más asociadas según la medida de información mutua son: \n")
for palabra_y_info_mutua in informaciones_mutuas.most_common(5):
    print("* {0} con información mutua: {1}".format(palabra_y_info_mutua[0], palabra_y_info_mutua[1]))

Los 5 pares de palabras más asociadas según la medida de información mutua son: 

* index value con información mutua: 1838.0320849868358
* conference news con información mutua: 1421.3935232619224
* 30 average con información mutua: 1384.3817325708626
* executive chief con información mutua: 1228.8735910193982
* fell index con información mutua: 1106.339421879901


## 3) Word embeddings, distancia semántica y Word-Net

### a) Utilizando el test WordSim353 , comparar el rendimiento entre LSA y Word2Vec.

#### Modelo Word2Vec y LSA

In [5]:

# Voy a ejecutar éste código solamente si no lo ejecute antes.
if not os.path.exists('obtener_metrica_de_similitud_por_modelo-()-{}.pickle'):
    
    # Primero el modelo Word2Vec.
    # Para utilizar éste modelo, se debe descargar el link puesto debajo y ubicarlo en la carpeta source del proyecto.
    # Para el model de Google: http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/
    word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
    
    # Luego, el modelo LSA.
    # Primero, tenemos que armar el corpus para LSA adaptándonos al formato TF-IDF.
    oraciones = []
    # Pasamos todos los documentos del corpus a la lista oraciones, limpiando en el proceso.
    directorio = os.fsencode('brown/')
    for archivo in os.listdir(directorio):
        filename = os.fsdecode(archivo)
        nombre_completo = 'brown/' + filename
        with open(nombre_completo) as f:
            lineas = f.read().splitlines()
            lineas_limpias = [" ".join(word.split("/")[0] for word in s.split()) for s in lineas]
            oraciones += lineas_limpias

    # Sacamos palabras que queremos ignorar, espacios vacíos, etc. 
    ignorables = list('for a of the and to in if from by'.split()) + list(punctuation) + list(whitespace) + [[]]

    # Armamos una gran lista de palabras por documento
    palabras = [[word for word in document.lower().split() if word not in ignorables]
                 for document in oraciones if document]

    lsa_dicionario = gensim.corpora.Dictionary(palabras)
    # doc2bow() cuenta la cantidad de apariciones de c/palabra, transforma la palabra a su entero_id y devuelve
    # el resultado como un vector esparso.
    lsa_corpus = [lsa_dicionario.doc2bow(palabra) for palabra in palabras]

    # Inicializamos modelo TF-IDF.
    modelo_tfidf = gensim.models.TfidfModel(lsa_corpus)
    corpus_tfidf = modelo_tfidf[lsa_corpus]
    if not os.path.exists('lsi_model'):
        # Finalmente, tenemos el modelo LSI (LSA).
        modelo_lsi = gensim.models.LsiModel(corpus_tfidf, id2word=lsa_dicionario, num_topics=300)
        modelo_lsi.save('lsi_model')
    else:
        modelo_lsi = gensim.models.LsiModel.load('lsi_model')

#### Testeo Similitud con la medida conocida como Cosine Similarity.

In [6]:
def obtener_similitud_lsa(palabra_1, palabra_2):
    doc = palabra_1 + " " + palabra_2
    similitud_query = lsa_dicionario.doc2bow(doc.lower().split())
    similitud_lsi = modelo_lsi[similitud_query]
    maximo = 0
    for tupla in similitud_lsi:
        if tupla[1] > maximo:
            maximo = tupla[1]
    return maximo

@load_or_call
def obtener_metrica_de_similitud_por_modelo():
    similitudes_por_modelo = {'WordSim353': {}, 'Word2Vec': {}, 'LSA': {}}
    similitud_test_filename = 'wordsimtest/wordsim_similarity_goldstandard.txt'
    with open(similitud_test_filename, 'rb') as similitud_test_file:
        for line in similitud_test_file.readlines():
            # Parseo línea de palabras similares.
            linea = line.decode('UTF-8').split()
            primera_palabra = linea[0]
            segunda_palabra = linea[1]
            subindice = primera_palabra + " " + segunda_palabra
            # Veo similaridad en WordSim353.
            similitudes_por_modelo['WordSim353'][subindice] = linea[2]
            # Veo similarity en Word2Vec.
            similitudes_por_modelo['Word2Vec'][subindice] = word2vec_model.wv.similarity(primera_palabra, segunda_palabra)
            # ToDo: agregar la parte de LSA
            similitudes_por_modelo['LSA'][subindice] = obtener_similitud_lsa(primera_palabra, segunda_palabra)
            return similitudes_por_modelo

In [7]:
similitudes_por_modelo = obtener_metrica_de_similitud_por_modelo()

In [8]:
print("La similitud para Word2Vec en el caso de tiger cat es: {0}".format(similitudes_por_modelo['Word2Vec']['tiger cat']))

La similitud para Word2Vec en el caso de tiger cat es: 0.5172961902020222


In [9]:
print("La similitud para LSA en el caso de tiger cat es: {0}".format(similitudes_por_modelo['LSA']['tiger cat']))

La similitud para LSA en el caso de tiger cat es: 0.004632709012058887


Recortamos el input de similitudes para que no ensucie el documento. En general, y como se aprecia en el ejemplo, el modelo Word2Vec presenta resultados con similitudes más altas en sus conjuntos de palabras comparado con el modelo LSA. Creemos que ésto se debe a que el set de entrenamiento del modelo Word2Vec provisto por Google es mucho más robusto y amplio que el brown corpus de LSA (que data de 1961).